In [13]:
import cv2
import mediapipe as mp
import numpy as np
import time


In [14]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize MediaPipe Drawing Utilities
mp_drawing = mp.solutions.drawing_utils


In [15]:
# Start video capture (webcam)
cap = cv2.VideoCapture(0)

# Create an empty canvas (white background)
canvas = np.ones((480, 640, 3), dtype=np.uint8) * 255  # white canvas


In [16]:
#  Function to draw on canvas based on hand movement
def draw_on_canvas(landmarks, canvas, color=(0, 0, 0), radius=5):
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    x, y = int(index_tip.x * canvas.shape[1]), int(index_tip.y * canvas.shape[0])
    cv2.circle(canvas, (x, y), radius, color, -1)

# Function to erase on canvas based on hand gesture (open hand)
def erase_on_canvas(landmarks, canvas, radius=15):
    for id, lm in enumerate(landmarks):
        x, y = int(lm.x * canvas.shape[1]), int(lm.y * canvas.shape[0])
        cv2.circle(canvas, (x, y), radius, (255, 255, 255), -1)  # erase by painting white

# Function to check if the thumb is up
def is_thumb_up(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_base = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC]
    
    # Check if the thumb tip is above the thumb base (thumbs-up gesture)
    if thumb_tip.y < thumb_base.y:
        return True
    return False

In [17]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a later mirror effect
    frame = cv2.flip(frame, 1)

    # Convert frame to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame and get hand landmarks
    results = hands.process(rgb_frame)

    # Draw landmarks and detect gestures
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the tip of the index finger (landmark 8)
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

            # Detect drawing gesture by checking index finger tip position
            if index_tip.y < thumb_tip.y:  # index finger tip above thumb tip (simple condition for drawing)
                draw_on_canvas(hand_landmarks.landmark, canvas, color=(0, 0, 0))

            # Detect erasing gesture (open hand)
            elif index_tip.y > thumb_tip.y:  # Use thumb and index to detect open hand
                erase_on_canvas(hand_landmarks.landmark, canvas)

            # Draw landmarks on the frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the canvas and frame
    cv2.imshow('Virtual Whiteboard', canvas)
    cv2.imshow('Hand Tracking', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

In [18]:
cap.release()
cv2.destroyAllWindows()